In [1]:
import tensorflow.keras as keras

In [5]:
import dataclasses
import tempfile
from dataclasses import dataclass, field
from typing import List, Dict, Any, Type

import tensorflow as tf
import tensorflow.keras as keras
from tensorboard.plugins.hparams import api as hp
from typing import Tuple
import math

In [6]:
class Embedding(keras.layers.Layer):

    def __init__(self, in_dim: int, out_dim: int, l1_l2: Tuple[float, float], init_scale: float,
                 aggregation: str, activation: str, use_bias: bool):
        super(Embedding, self).__init__()
        self._config = {
            'in_dim': in_dim,
            'out_dim': out_dim,
            'l1_l2': l1_l2,
            'init_scale': init_scale,
            'aggregation': aggregation,
            'activation': activation,
            'use_bias': use_bias,
        }

        stddev: float = init_scale * math.sqrt(1.0 / in_dim)
        self._w = self.add_weight(
            initializer=tf.keras.initializers.TruncatedNormal(stddev=stddev), shape=(in_dim, out_dim), trainable=True,
            regularizer=tf.keras.regularizers.l1_l2(*l1_l2), dtype=tf.float32, name='fieldunaware_embedding_w'
        )
        if use_bias:
            self._b = self.add_weight(
                initializer='zeros', shape=(1, out_dim), trainable=True, dtype=tf.float32, name='fieldunaware_embedding_b'
            )
        else:
            self._b = None
        self._aggregation = dict(
            mean=tf.sparse.segment_mean,
            sqrt_n=tf.sparse.segment_sqrt_n,
            sum=tf.sparse.segment_sum,
        )[aggregation]
        self._activation = dict(relu=tf.nn.relu, tanh=tf.nn.tanh, identity=tf.identity)[activation]

    def call(self, inputs, **kwargs):
        # squeeze batch dimension, assuming that the given batch size here is always 1
        indexes: tf.Tensor = tf.reshape(inputs[0], shape=[tf.size(inputs[0])])
        values: tf.Tensor = tf.reshape(inputs[1], shape=[tf.size(inputs[1])])
        # add back batch dimension
        embedding_sum = tf.expand_dims(self._aggregation(self._w, segment_ids=indexes, indices=values), axis=0)
        if self._b is not None:
            return self._activation(embedding_sum + self._b)
        else:
            return self._activation(embedding_sum)

    def get_config(self):
        return self._config


In [9]:
class AUCFromLogit(keras.metrics.AUC):
    def update_state(self, y_true, y_pred, sample_weight=None):
        return super().update_state(y_true, tf.nn.sigmoid(y_pred), sample_weight)

In [10]:
single_feature_num = 5
indexes = keras.Input(shape=[None], dtype=tf.int32, batch_size=1, name='indexes')
values = keras.Input(shape=[None], dtype=tf.int32, batch_size=1, name='values')

# create logistic regression model
# using FieldUnawareEmbedding layer with outdim=1 to reduce computational complexity
# other than dense kernel initialization, the model structure is exactly same as SimpleLRNaive
logit = Embedding(in_dim=single_feature_num, out_dim=1, l1_l2=(0., 0.),
                  init_scale=1.0, aggregation='sum', activation='identity', use_bias=True)([indexes, values])
# squeeze 3rd dimension and reshape it to [1, batch size]
logit = tf.squeeze(logit, axis=[2])

# Note that, sigmoid is applied by Java inferrer layer at inference time
model = keras.Model(inputs=[indexes, values], outputs=logit)

In [12]:
learning_rate=0.001
loss_object = keras.losses.BinaryCrossentropy(from_logits=True)

# get optimizer from hyper parameter setting
optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)

# compile model with optimization and metric configs to build a graph
model.compile(
    loss=loss_object, optimizer=optimizer,
    metrics=[AUCFromLogit(name='auc')]
)

TypeError: To be compatible with tf.contrib.eager.defun, Python functions must return zero or more Tensors; in compilation of <function AUCFromLogit.update_state at 0x13e86c9d8>, found return value of type <class 'tensorflow.python.framework.ops.Operation'>, which is not a Tensor.